# Execute batch groundness evaluation flow using Promptflow Python SDK 

### Overview
Prompt flow is a suite of development tools designed to streamline the end-to-end development cycle of LLM-based AI applications, from ideation, prototyping, testing, evaluation to production deployment and monitoring. It makes prompt engineering much easier and enables you to build LLM apps with production quality. 

In this handson, you will be able to:
Evaluate your flows, calculate quality and performance metrics with run result datasets.
Debug and iterate your flows, especially tracing interaction with LLMs with ease.
In order to calculate the other metrics like accuracy, relevance score. Please refer to [Develop evaluation flow](https://microsoft.github.io/promptflow/how-to-guides/develop-a-dag-flow/develop-evaluation-flow.html) to learn how to develop an evaluation flow.


#### 1. Create Promptflow client with Credential and configuration
#### 2. AI Studio batch run to get the base run data 
#### 3. Run Groundedness Evaluation of the Promptflow 


[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


In [49]:
import json
import os
import time

# Import required libraries
from promptflow.azure import PFClient
from promptflow.entities import Run
# Import required libraries
from azure.identity import DefaultAzureCredential, EnvironmentCredential, InteractiveBrowserCredential
from dotenv import load_dotenv
from azure.core.exceptions import HttpResponseError

load_dotenv("../../.env")

with open('../3_2_prototyping/config.json', 'r') as f:
    config = json.load(f)
    
print(config["subscription_id"])
print(config["resource_group"])
print(config["workspace_name"]) # Azure AI Studio project name which is not the same as the Azure ML workspace name

print(os.environ["AZURE_CLIENT_ID"])
print(os.environ["AZURE_CLIENT_SECRET"])
print(os.environ["AZURE_TENANT_ID"])


3d4d3dd0-79d4-40cf-a94e-b4154812c6ca
slm-innovator-rg
slm-pjt1
7ac00d14-31c3-4eac-a9a9-338f6dbd1c98
-Xc8Q~KelkSWjjbjxY6VXZ6Ii40x0DDTL3knsbZx
16b3c013-d300-468d-ac64-7eda0820b6d3


In [46]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_status(pf_azure_client:PFClient, run_result:Run):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = pf_azure_client.runs.get(run_result).status
        if status == "Preparing":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = pf_azure_client.runs.get(run_result).status
        pbar.update(1)
        print("Promptflow Running Completed")

## 1. Create Promptflow client with Credential and configuration
- Create a promptflow client with the credential and configuration. You need to set the `config.json` file with subscription_id, resource_group and workspace_name

In [51]:

credential = DefaultAzureCredential()
# if you cannot use DefaultAzureCredential, you need to set up the environment variables for the service principal
# credential = EnvironmentCredential(
#     client_id=os.environ["AZURE_CLIENT_ID"],
#     secret=os.environ["AZURE_CLIENT_SECRET"],
#     tenant=os.environ["AZURE_TENANT_ID"],
# )
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")

pf_azure_client = PFClient.from_config(credential=credential, path="../3_2_prototyping/config.json")

# pf_azure_client = PFClient(credential=credential, 
#                            subscription_id="your subscription id", 
#                            resource_group_name="your resource group name", 
#                            workspace_name="your workspace name")            

try:
    workspace = pf_azure_client.ml_client.workspaces.get(name=config["workspace_name"])
    print(f"Connected to Azure AI Studio Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")


Found the config file in: ../3_2_prototyping/config.json


Connected to Azure AI Studio Workspace: slm-pjt1
Workspace Location: eastus
Workspace ID: /subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


## 2. AI Studio batch run to get the base run data 

In [57]:
flow_path = "../3_2_prototyping/chat-serverless"
data_path = "../3_2_prototyping/data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('../3_2_prototyping/data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

base_run = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_serverless_context_data",
    tags={"chat_serverless_context_jsonl": "", "1st_round": ""},
)

[2024-10-24 10:23:43 +0000][promptflow][WARNING] - You're using compute session, if it's first time you're using it, it may take a while to build session and you may see 'NotStarted' status for a while. 
[2024-10-24 10:23:43 +0000][promptflow][WARNING] - The trace Cosmos DB for current workspace/project is not ready yet, your traces might not be logged and stored properly.
To enable it, please run `pf config set trace.destination=azureml://subscriptions/<subscription-id>/resourceGroups/<resource-group-name>/providers/Microsoft.MachineLearningServices/workspaces/<workspace-or-project-name>`, prompt flow will help to get everything ready.



Portal url: https://ai.azure.com/projectflows/trace/run/chat_serverless_variant_0_20241024_102342_227780/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [58]:
monitor_status(pf_azure_client, base_run)

Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:00<00:01,  1.84step/s]

Current Status: Preparing
Current Status: Preparing


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [00:21<00:12, 12.55s/step]

Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:42<00:00, 14.16s/step]

Promptflow Running Completed


In [59]:
detail = pf_azure_client.get_details(base_run)

detail

,inputs.question,inputs.context,inputs.line_number,outputs.phi35_answer,outputs.gpt4o_answer
outputs.line_number,,,,,
0,tell me about your TrailMaster X4 Tent,TrailMaster X4 Tent is a durable polyester ten...,0,TrailMaster X4 Tent은 강력한 플레이너와 동반하여 야외 활동에 필수...,"TrailMaster X4 텐트는 초경량 백패킹 텐트로, 4인용입니다. 설치가 간편..."
1,Do you have any climbing gear?,TrailMaster X4 Tent is a durable polyester ten...,1,"네, 탐슨 제품도 있습니다. 예를 들어, 랜덤 등반 튜브, 안전 포크, 그리고 안...",아쉽게도 제가 등산 장비를 직접 갖고 있진 않아요. 😅 하지만 당신이 찾는 장비에 ...
2,Can you tell me about your selection of tents?,TrailMaster X4 Tent is a durable polyester ten...,2,"트레일마스터 X4 텐트는 강성 폴리에스터로 만들어진 4인용 텐트로, 물 방지 구조...",저희 텐트 종류에 대해 말씀드릴게요! ⛺\n\n1. **캐빈 텐트**: 가족 여행에...
3,Do you have TrekReady Hiking Boots? How much i...,TrailMaster X4 Tent is a durable polyester ten...,3,트레크리어 하이킹 보를 가지고 있습니다. 가격은 필요한 세부 정보로 확인해 주세요...,TrekReady 하이킹 부츠에 대해 말씀드릴게요! 😄\n\n현재 TrekReady...
4,can you tell me BaseCamp Folding Table?,TrailMaster X4 Tent is a durable polyester ten...,4,BaseCamp Folding Table는 약 48x24인치의 가벼운 알루미늄 템...,BaseCamp 접이식 테이블은 캠핑이나 야외 활동에 딱 맞는 테이블이에요. 가볍고...


## 3. Run Groundedness Evaluation of the Promptflow 
The eval-groundness flow is illustrating measures how grounded the model's predicted answers are against the context. Even if LLM’s responses are true, if not verifiable against context, then such responses are considered ungrounded.

> 🧪 +For Your Information<br>
> **Groundedness** is a measure of how well the model's responses are grounded in the context. A grounded response is one that is directly supported by the context. For example, if the context is about a dog, a grounded response would be "Dogs are mammals." An ungrounded response would be "Dogs can fly."

In [79]:
import datetime

eval_groundedness_flow_path = "./evaluation/"
data_path = "./data/qna_outdoor.jsonl"

with open('../3_2_prototyping/data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping={
        "question": "${data.question}",
        "context": context.get("context")    ,
        "answer": "${run.outputs.gpt4o_answer}",
    }
eval_name = "eval_groundedness"
now = datetime.datetime.now()
timestamp = now.strftime("%m_%d_%H%M")
eval_name = str(eval_name + "_" + timestamp)

eval_groundedness_result = pf_azure_client.run(
    flow=eval_groundedness_flow_path,
    data=data_path,
    run=base_run,  # use run as the variant
    column_mapping=column_mapping,
    display_name=eval_name,
    name=eval_name,
)



# pf_azure_client.stream(eval_groundedness_result)

[2024-10-24 10:55:00 +0000][promptflow][WARNING] - You're using compute session, if it's first time you're using it, it may take a while to build session and you may see 'NotStarted' status for a while. 
[2024-10-24 10:55:00 +0000][promptflow][WARNING] - The trace Cosmos DB for current workspace/project is not ready yet, your traces might not be logged and stored properly.
To enable it, please run `pf config set trace.destination=azureml://subscriptions/<subscription-id>/resourceGroups/<resource-group-name>/providers/Microsoft.MachineLearningServices/workspaces/<workspace-or-project-name>`, prompt flow will help to get everything ready.



Portal url: https://ai.azure.com/projectflows/trace/run/eval_groundedness_10_24_1054/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [80]:
monitor_status(pf_azure_client, eval_groundedness_result)

Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:00<00:00,  2.50step/s]

Current Status: Preparing
Current Status: Preparing


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [00:21<00:12, 12.48s/step]

Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:31<00:00, 10.65s/step]

Promptflow Running Completed


In [81]:
detail = pf_azure_client.get_details(eval_groundedness_result)

detail

,inputs.question,inputs.context,inputs.answer,inputs.line_number,outputs.gpt_groundedness
outputs.line_number,,,,,
0,Can you tell me about your selection of tents?,TrailMaster X4 Tent is a durable polyester ten...,"TrailMaster X4 텐트는 초경량 백패킹 텐트로, 4인용입니다. 설치가 간편...",0,1.0
1,can you tell me BaseCamp Folding Table?,TrailMaster X4 Tent is a durable polyester ten...,아쉽게도 제가 등산 장비를 직접 갖고 있진 않아요. 😅 하지만 당신이 찾는 장비에 ...,1,1.0
2,Do you have any climbing gear?,TrailMaster X4 Tent is a durable polyester ten...,저희 텐트 종류에 대해 말씀드릴게요! ⛺\n\n1. **캐빈 텐트**: 가족 여행에...,2,1.0
3,Do you have TrekReady Hiking Boots? How much i...,TrailMaster X4 Tent is a durable polyester ten...,TrekReady 하이킹 부츠에 대해 말씀드릴게요! 😄\n\n현재 TrekReady...,3,1.0
4,tell me about your TrailMaster X4 Tent,TrailMaster X4 Tent is a durable polyester ten...,BaseCamp 접이식 테이블은 캠핑이나 야외 활동에 딱 맞는 테이블이에요. 가볍고...,4,5.0
